<h1><center>RQ2</center></h1>
<center>Marcel de Cabanyes Torras</center>
<center> 207526</center>

## Preparation 
Here we are going to generate the cluster as in RQ1, reffer to that notebook to see in detail the process and reasoning behind.

In [1]:
import os.path
import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(parent_dir, 'search-engine/')) #Load utils.py from search-engine/
from utils import *
import pandas as pd
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string
from sklearn.manifold import TSNE
import nltk
nltk.download('punkt') # used in sent_tokenize
from sklearn.cluster import KMeans, DBSCAN
pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package stopwords to /home/marcel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/marcel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Load data and generate index.

In [2]:
data_dir = os.path.join(parent_dir, 'search-engine/data') #Load data from search-engine/data
start_time = time.time()

df_tweets_simp = preprocess(data_dir) # Preprocess dataset

print("Total time to preprocess dataset of {} tweets: {} seconds." .format(len(df_tweets_simp),np.round(time.time() - start_time,2)))

start_time = time.time()
print("\n======================\nCreating index...")
numTweets = len(df_tweets_simp)
myIndex, df, idf = create_index_tfidf(df_tweets_simp,numTweets) # Create indexes
print("Total time to create the index: {} seconds." .format(np.round(time.time() - start_time,2)))


Loading dataset...

Preprocessing dataset...
Total time to preprocess dataset of 100100 tweets: 54.11 seconds.

Creating index...
10001 tweets have been indexed in  2.36 seconds.
20001 tweets have been indexed in  4.68 seconds.
30001 tweets have been indexed in  7.8 seconds.
40001 tweets have been indexed in  10.37 seconds.
50001 tweets have been indexed in  12.62 seconds.
60001 tweets have been indexed in  14.85 seconds.
70001 tweets have been indexed in  17.05 seconds.
80001 tweets have been indexed in  19.37 seconds.
90001 tweets have been indexed in  21.63 seconds.
100001 tweets have been indexed in  23.86 seconds.
Creating df and idf...
Total time to create the index: 24.44 seconds.


## Clustering
### (Question a)

The next cells do all the process of clustering as in RQ1, reffer to it for more detail.

In [3]:
# Functions for vector embedding        

# Splits into sentences
def get_sentences(tweets, stops, verbose=10000):
    #loop over all docs (paragraphs in our case)
    lines = len(tweets)
    for i, tweet in enumerate(tweets):        
        # use nltk.sent_tokenize to split paragraphs into sentences
        for s in nltk.sent_tokenize(tweet):
            # preprocess each sentence using gensim (return string not list)
            yield ' '.join(getTerms(s, stops))
        # print progress if needed
        if verbose > 0 and (i + 1) % verbose == 0:
            print("Progress: {}%.".format(np.round((i/lines)*100,2)))
    print("Progress: 100%.") 

# Generates w2v model
def create_w2v_model(df_tweets, stops):
    # Get texts as list
    tweets = list(df_tweets['text'])
    # Get sentences
    sentences = list(get_sentences(tweets,stops))
    # Split in words
    words = [s.split() for s in sentences ]
    #Create model
    print("\n======================\nGenerating w2v model...")
    w2v_model = Word2Vec(sentences = words, size = 100, window = 10, min_count = 10, negative = 15, sg = 0)
    return w2v_model

# Transforms set of terms (tweet) into vector (averaging the terms vectors)
def get_vector(w2v_model, terms):
    vector = 0
    count = 0
    for term in terms: # Gets the vector for each term of the tweet
        if w2v_model.wv.__contains__(term):
            vector += w2v_model.wv.__getitem__(term)
            count += 1
    avg_vector = vector/count # Averages it
    #norm_vector = avg_vector / np.linalg.norm(avg_vector) # Normalizes it
    return avg_vector

def w2v_rank(top, myQuery, stops, w2v_model, result_tweets):
    query_terms = getTerms(myQuery, stops) # Get query terms
    query_vector = get_vector(w2v_model, query_terms) # Get query vector from model
    top_20 = topper(top) # Class defined in utils to retain the topX values
    for i, tweet in result_tweets.iterrows():
        tweet_vector = get_vector(w2v_model,tweet['terms']) # Get tweet vector
        score = np.dot(query_vector, tweet_vector)  # Compute the score as the dot product
        top_20.append(tweet['id'], score)
    top_20.sort()
    rankedTweets = top_20.to_dataframe('id', 'score')
    rankedTweets = pd.merge(rankedTweets,result_tweets, on='id',how='left')
    return rankedTweets

Generate w2v model.

In [4]:
stops = set(stopwords.words('english'))
w2v_model = create_w2v_model(df_tweets_simp, stops)
df_tweets_simp_red = df_tweets_simp[:20000]

Progress: 9.99%.
Progress: 19.98%.
Progress: 29.97%.
Progress: 39.96%.
Progress: 49.95%.
Progress: 59.94%.
Progress: 69.93%.
Progress: 79.92%.
Progress: 89.91%.
Progress: 99.9%.
Progress: 100%.

Generating w2v model...


Embed tweets.

In [5]:
print("\n======================\nEmbedding tweets...")
count = 0
lines = len(df_tweets_simp_red)
list_embedded_tweets = [] # this list of indexes will help us later to link the tweet to its vector
for i, tweet in df_tweets_simp_red.iterrows():
    try:
        tweet_vector = get_vector(w2v_model,tweet['terms']) # get vector representation
        if count == 0:
            tweets_array = np.array(tweet_vector) # if its the first tweet, initialize the matrix
            list_embedded_tweets.append(i)
            count += 1
        else:
            tweets_array = np.vstack((tweets_array, np.array(tweet_vector))) # if not, stack it
            count += 1
            list_embedded_tweets.append(i)

    except:
        continue
    if(i % 2000 == 0):
        print("{}% of tweets have been converted to vectors.".format(round((i/lines)*100,2)))
print("100% of tweets have been converted to vectors.")


Embedding tweets...
0.0% of tweets have been converted to vectors.
10.0% of tweets have been converted to vectors.
20.0% of tweets have been converted to vectors.
30.0% of tweets have been converted to vectors.
40.0% of tweets have been converted to vectors.
50.0% of tweets have been converted to vectors.
60.0% of tweets have been converted to vectors.
70.0% of tweets have been converted to vectors.
80.0% of tweets have been converted to vectors.
90.0% of tweets have been converted to vectors.
100% of tweets have been converted to vectors.


Embed into 2 dimensional space.

In [6]:
print("\n======================\nEmbedding into 2 dimensional space...")
X_embedded = TSNE(n_components=2, verbose=2).fit_transform(tweets_array)


Embedding into 2 dimensional space...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 19912 samples in 0.068s...
[t-SNE] Computed neighbors for 19912 samples in 78.633s...
[t-SNE] Computed conditional probabilities for sample 1000 / 19912
[t-SNE] Computed conditional probabilities for sample 2000 / 19912
[t-SNE] Computed conditional probabilities for sample 3000 / 19912
[t-SNE] Computed conditional probabilities for sample 4000 / 19912
[t-SNE] Computed conditional probabilities for sample 5000 / 19912
[t-SNE] Computed conditional probabilities for sample 6000 / 19912
[t-SNE] Computed conditional probabilities for sample 7000 / 19912
[t-SNE] Computed conditional probabilities for sample 8000 / 19912
[t-SNE] Computed conditional probabilities for sample 9000 / 19912
[t-SNE] Computed conditional probabilities for sample 10000 / 19912
[t-SNE] Computed conditional probabilities for sample 11000 / 19912
[t-SNE] Computed conditional probabilities for sample 12000 / 19912
[t-SNE] Co

Cluster the tweets using the optimal number of clusters (seen in RQ1).

In [7]:

print("\n======================\nClustering...")
kmeans = KMeans(n_clusters=12, max_iter= 3000)
kmeans.fit(X_embedded)

y_kmeans = kmeans.predict(X_embedded)

df_tweets_clustered = df_tweets_simp_red.copy().loc[list_embedded_tweets] 
df_tweets_clustered['cluster'] = y_kmeans # Map cluster to original tweet


Clustering...


In [8]:
df_tweets_clustered.head()

,likes,retweets,hashtags,date,user,id,text,url,terms,cluster
0,1085,565,[],Sat Nov 21 15:19:49 +0000 2020,23560243-596b-501a-8c17-ce8454fde1c1,1330169051222597645,RT @89351f88-728f-5e9a-a481-45513502cb51: Case in point: @02e6fb19-6e6e-5b3a-977b-14c8000d52d1,https://twitter.com/i/web/status/1330169051222597645,"[89351f88728f5e9aa48145513502cb51, case, point, 02e6fb196e6e5b3a977b14c8000d52d1]",2
1,0,0,[],Sat Nov 21 15:19:49 +0000 2020,1ae5add9-83ea-5a19-90fe-eab48de8c501,1330169051306405889,@02e6fb19-6e6e-5b3a-977b-14c8000d52d1 CAn we keep him in quarantine for the next month or so and take away his phone please?,https://twitter.com/i/web/status/1330169051306405889,"[02e6fb196e6e5b3a977b14c8000d52d1, keep, quarantin, next, month, take, away, phone, pleas]",6
2,0,0,[],Sat Nov 21 15:19:49 +0000 2020,eb71fc65-45e0-55de-ad32-c7f8bc4a950b,1330169050870280194,RT @c500f6b6-cec5-576b-98d0-6714c6f533ea: My dad died today in a nursing home. \n\nHe was tested for Covid 3 times over the last 3 weeks. \n\nHe was negative every tim…,https://twitter.com/i/web/status/1330169050870280194,"[c500f6b6cec5576b98d06714c6f533ea, dad, die, today, nurs, home, test, covid, 3, time, last, 3, week, neg, everi, tim]",7
3,0,0,[],Sat Nov 21 15:19:49 +0000 2020,e7bbce28-564c-51c8-96e6-314bf2c1a5b6,1330169051474239491,"RT @403e43ea-81b6-5f1b-b65c-1ab0c0ec6632: 📍DON’T CHASE SWEDISH HERD STRATEGY. As many know, 🇸🇪 took the careless “herd” strategy in spring to let the coronavirus run…",https://twitter.com/i/web/status/1330169051474239491,"[403e43ea81b65f1bb65c1ab0c0ec6632, dont, chase, swedish, herd, strategi, mani, know, took, careless, herd, strategi, spring, let, coronaviru, run]",6
4,0,0,[BTS],Sat Nov 21 15:19:49 +0000 2020,9f1f2dee-b578-54fc-bcca-f8bbb2e45473,1330169051449012227,"RT @c6baa608-68fc-5709-8e87-2d65015fa9c0: [기사] #BTS – ‘BE (Deluxe Edition)’ review: a sensitive, stunning document of pandemic life\nhttps://t.co/4afIKhcOQC",https://twitter.com/i/web/status/1330169051449012227,"[c6baa60868fc57098e872d65015fa9c0, 기사, bt, delux, edit, review, sensit, stun, document, pandem, life, httpstco4afikhcoqc]",9


## Diversity score
### (Question b)

For our diversfication score we want to take into account to factors:

    - Coverage (number of distinct clusters in the ranking)
    - The inverse of the underliying distribution (inverse of number of tweets by cluster)
    
That is, we are going to maximize the number of clusters included in our ranking, emphasizing those that normally have less visibility. So our score for a rank `R` will be:

    diversity_score(R) = Σ(1 - perc(C_t))*coverage(R)
    
For each tweet `t` in the ranking, we compute `1 - perc(C_t)`, where `perc(C_t)` is the number of tweets in the cluster of `t` divided by the number of tweets. We sum the results, and multiply them by the coverage of the ranking (`coverage(R)`). 

## Diversified ranking
### (Question c)

### xQuAD

It is a greedy algorithm that iteratively selects a new document, and pushes it into the current solution.  The selection  process  consists  in  choosing  each  time  the  document that maximizes the following equation:

    (1−λ)·P(d|q) + λ·P(d, ̄S|q)

Such formula combines the probability for a documentd of being relevant for a query q, i.e.,P(d|q) and the diversity of a candidate document d, respectively. At each step, the algorithm updates the coverage degree of solution S for each candidate document, then it scans R\S in order to choose the best document.

In our case, we are going to interpret `P(d|q)` as TF-IDF of the ranking and `P(d, ̄S|q)` as our diversity score. So for each possible ranking including the new tweet and excluding another one, we are going to compute the above formula. Note that `λ` represents a tradeoff between the two scores.

So we firsly define a dictionary cluster to `(1 - perc(C))`.

In [9]:
cluster_to_inv_perc_dict = {}
# count the number of tweets per cluster
cluster_count_df = df_tweets_clustered.groupby('cluster', as_index = False).count()[['cluster','id']]
# compute percentage
cluster_count_df['perc'] =  cluster_count_df['id']/sum(cluster_count_df['id'])
for i, row in cluster_count_df.iterrows():
        cluster_to_inv_perc_dict[row['cluster']] = 1 - row['perc'] # inverse percentage

In [10]:
cluster_to_inv_perc_dict

{0.0: 0.9286359983929289,
 1.0: 0.9145239051828044,
 2.0: 0.9358175974286862,
 3.0: 0.9242165528324628,
 4.0: 0.8985034150261149,
 5.0: 0.9258738449176376,
 6.0: 0.8918742466854158,
 7.0: 0.9364704700683005,
 8.0: 0.9178384893531539,
 9.0: 0.9198473282442748,
 10.0: 0.9184913619927682,
 11.0: 0.887906789875452}

Now we are going to map each tweet id to its cluster and cluster score.

In [11]:
tweet_id_to_cluster_dict = {}
for i, row in df_tweets_clustered.iterrows():
    tweet_id_to_cluster_dict[row['id']] = (row['cluster'], cluster_to_inv_perc_dict[row['cluster']])

In [12]:
def coverage(rank_clusters):
    return len(set(rank_clusters)) # return the coverage of a list of clusters

def diversity_score(tweet_id, tf_idf, top_20, tweet_id_to_cluster_dict ,l):
    
    '''
    For each tweet into the current top, try to excluding it from the top and including the new candidate,
    and retain the top with the highest (diversity) score. 
    '''   
    
    # compute the current score
    curr_cluster_list = [tweet_id_to_cluster_dict[tupla[0]][1] for tupla in top_20] # clusters in the curr top
    curr_cov = coverage(curr_cluster_list) # coverage of the curr top
    curr_dist = sum([tweet_id_to_cluster_dict[tupla[0]][0] for tupla in top_20]) # sum of inverse dist
    curr_tf_idf = sum([tupla[1] for tupla in top_20]) # sum of curr tf-idf
    curr_score =  (1 - l)*curr_tf_idf + l*(curr_cov)*curr_dist # diversity score computation

    best_score = curr_score # set the best score to the current one
    best_top_20 = top_20 # set the best top to the current one

    for tweet, tf_idf_score in top_20: # for each tweet in the current top
        
        copy = top_20.copy() # copy the current top
        copy.remove((tweet, tf_idf_score)) # remove the tweet in questio from the top
        copy.append((tweet_id, tf_idf)) # include the new candidate
        cluster_list = [tweet_id_to_cluster_dict[tupla[0]][1] for tupla in copy] # get the clusters ids...
        #...from the copy top
        copy_cov = coverage(cluster_list) # compute the coverage of the new top
        # sum the (1 - perc(C)) of all the tweets included into the top
        copy_dist = sum([tweet_id_to_cluster_dict[tupla[0]][0] for tupla in copy]) 
        copy_tf_idf = sum([tupla[1] for tupla in copy]) # sum the tf-idf of the new top
        new_score =  (1 - l)*copy_tf_idf + l*(copy_cov)*copy_dist # use our formula 
        if new_score >= best_score: # compare with the current best top and score, and retain the best one
            best_score = new_score
            best_top_20 = copy    

    return best_top_20 # return the new best top 

def divers_rank(query, tweets, index, idf, top,tweet_id_to_cluster_dict , l):
    
    # firstly we compute the tf-idf as in that ranking
    stops = set(stopwords.words('english'))
    terms = getTerms(query, stops)
    tweetVectors=defaultdict(lambda: [0]*len(terms)) 
    queryVector=[0]*len(terms)

    query_terms_count = collections.Counter(terms) 
    query_norm = la.norm(list(query_terms_count.values())) # get the norm of the query
    
    tweets_id = tweets['id'].tolist()

    for termIndex, term in enumerate(terms): 
        if term not in index:
            continue               
        queryVector[termIndex]=query_terms_count[term] * idf[term]        
        for tweetIndex, (tweet_id, tf) in enumerate(index[term]):
            if tweet_id in tweets_id:
                # get the terms of the tweet with its frequency and the norm
                tweet_terms_count= list(collections.Counter(tweets[tweets['id']==tweet_id]['terms'].to_list().pop()).values())
                tweet_norm = la.norm(tweet_terms_count) 
                tweetVectors[tweet_id][termIndex] = tf/tweet_norm * idf[term] # set into the position of the term vecot df*idf

    # now we are going to take into account the diversity
    top_20 = []
    curr_score = 0
    for tweet_id, curTweetVec in tweetVectors.items(): # for each tweet
        tf_idf = np.dot(curTweetVec, queryVector) # compute the tf_idf

        if len(top_20) < 20: # fill the top with the first 20 tweets
            top_20.append((tweet_id, tf_idf))
        else: # if the top is full, decide whether to include the new tweet into the top or not
            top_20 = diversity_score(tweet_id, tf_idf, top_20, tweet_id_to_cluster_dict,l)
    # finally sort by tf-idf using our class topper
    sorted_top = topper(top)
    for tweet_id, score in top_20:
        sorted_top.append(tweet_id, score)
    
    sorted_top.sort() # sort
    rankedTweets = sorted_top.to_dataframe('id', 'score') # covert to df
    rankedTweets = pd.merge(rankedTweets,tweets, on='id',how='left') # merge with info of df tweets
    
    return rankedTweets

Reuse our function for search.

In [13]:
def submit_search(top, index, df_tweets, myQuery):
    start_time = 0
    #Gets the result of the query
    result_tweets = pd.DataFrame()

    start_time = time.time()
    result_tweets = search(myQuery, index, df_tweets)
    if len(result_tweets) == 0:
        print("\n======================\nNo matches found.")
    return result_tweets

Search and rank by diversified ranking.

In [14]:
top = 20
l = 0.4 # lambda
query = 'trump'
result_tweets = submit_search(top, myIndex, df_tweets_clustered, query)
divers_ranked_tweets = divers_rank(query, result_tweets, myIndex, idf, top,tweet_id_to_cluster_dict , l)

In [15]:
divers_ranked_tweets

,id,score,likes,retweets,hashtags,date,user,text,url,terms,cluster
0,1330165437414432768,6.906574,1450,98,[],Sat Nov 21 15:05:28 +0000 2020,07834142-57cb-5655-8c9c-8e06deda5119,tRump Vaccine for the tRump Virus,https://twitter.com/i/web/status/1330165437414432768,"[trump, vaccin, trump, viru]",7
1,1330183654925070346,5.981268,1,0,[],Sat Nov 21 16:17:51 +0000 2020,3101f384-5d0b-5fed-9e3a-4c4021649828,trumps words. It is what it is. 🤣,https://twitter.com/i/web/status/1330183654925070346,"[trump, word]",3
2,1330183695333003266,4.368101,1019,641,[],Sat Nov 21 16:18:01 +0000 2020,3bddad1d-62a7-59f5-930a-1420bf398d6e,"RT @fa75fa83-40e9-5478-80f7-6881a399c373: After meeting w/ Pres. Trump, Michigan Republicans—including Speaker @3fe5b04c-8cda-527c-bbc3-b414d7a8651d—closed down the Trump Hotel bar last…",https://twitter.com/i/web/status/1330183695333003266,"[fa75fa8340e9547880f76881a399c373, meet, w, pre, trump, michigan, republicansinclud, speaker, 3fe5b04c8cda527cbbc3b414d7a8651dclos, trump, hotel, bar, last]",0
3,1330184229150470145,3.453287,371,34,[],Sat Nov 21 16:20:08 +0000 2020,d591ebc7-be3c-5cf0-b387-f9fe39799921,This will upset Trump more than a quarter of a million dead Americans.,https://twitter.com/i/web/status/1330184229150470145,"[upset, trump, quarter, million, dead, american]",10
4,1330181198845534212,3.197122,0,0,[],Sat Nov 21 16:08:06 +0000 2020,6d4ad072-6769-53f3-b6fd-4495fee2f81a,G20 summit on the pandemic and Trump is golfing. Ok bitch,https://twitter.com/i/web/status/1330181198845534212,"[g20, summit, pandem, trump, golf, ok, bitch]",11
5,1330183634532392960,3.197122,0,0,[butitsnotdumbass],Sat Nov 21 16:17:46 +0000 2020,02418b58-a098-524a-b8ec-f01091097cb8,@efbf20c9-674d-5add-9500-16ba9f92bab3 #butitsnotdumbass Trump didn’t spread the coronavirus. The lockdown did.,https://twitter.com/i/web/status/1330183634532392960,"[efbf20c9674d5add950016ba9f92bab3, butitsnotdumbass, trump, didnt, spread, coronaviru, lockdown]",11
6,1330180371342888963,2.990634,544,60,[TrumpsKillerCovidVirus],Sat Nov 21 16:04:48 +0000 2020,1380348d-27a0-51a4-b796-90d74bfa8549,Trump has done everything in his power to keep this\n#TrumpsKillerCovidVirus\nin a mass spread...,https://twitter.com/i/web/status/1330180371342888963,"[trump, done, everyth, power, keep, trumpskillercovidviru, mass, spread]",6
7,1330165427499118600,2.819597,0,0,[],Sat Nov 21 15:05:25 +0000 2020,6159500a-4101-5e7e-bea0-90362d88b04d,"@c01b03d2-bfd1-5482-8b74-6792580daa6f @f91dc4ec-2fe0-505f-a048-da84e978dfeb Since YOU are an expert on how the pandemic should have been handled, what should Trump have done?… https://t.co/eysT811GwW",https://twitter.com/i/web/status/1330165427499118600,"[c01b03d2bfd154828b746792580daa6f, f91dc4ec2fe0505fa048da84e978dfeb, sinc, expert, pandem, handl, trump, done, httpstcoeyst811gww]",4
8,1330173916631998466,2.674904,0,0,[],Sat Nov 21 15:39:09 +0000 2020,fc5aaac0-f76a-53c7-9cc2-92bd93184e60,RT @1419f055-425b-5199-83fe-da3d98c37dcc: Trump and the Canadian Conservatives are intentionally interfering with controlling the spread of Covid-19. It’s sickening…,https://twitter.com/i/web/status/1330173916631998466,"[1419f055425b519983feda3d98c37dcc, trump, canadian, conserv, intent, interf, control, spread, covid19, sicken]",11
9,1330164593268830209,2.674904,0,0,"[WearAMaskSaveALife, COVID19]",Sat Nov 21 15:02:06 +0000 2020,a749fd93-ddd4-5d1a-b085-9ff73efa8114,"RT @bb733d3a-cf28-55fe-b9f5-d80af94a2d84: All The Cities Trump Had Rallies In, Are Now Having \nThe Greatest Spikes In Cases..\n#WearAMaskSaveALife\n#COVID19 #Supe…",https://twitter.com/i/web/status/1330164593268830209,"[bb733d3acf2855feb9f5d80af94a2d84, citi, trump, ralli, greatest, spike, case, wearamasksavealif, covid19, supe]",1


## Testing our new ranking
### RQ 2A

Define a set of queries and the columns that we are going to store.

In [16]:
myQueries = ['covid',
            'mask',
            'trump']
top = 20
columns = ['query', 'id', 'text', 'user', 'likes', 'retweets', 'hashtags', 'cluster', 'score']
output_dir = os.path.join(parent_dir, 'other-outputs/')

Reuse our function to rank (to rank by tf-idf).

In [17]:
def rank(top, result_tweets, index, idf, myQuery, method):

    methods_dict = {"1":tfidf_rank,"2":popularity_rank}
    selection = method
    ranked_tweets = pd.DataFrame()
    start_time = 0
    start_time = time.time()
    ranked_tweets = methods_dict[selection](myQuery, result_tweets, index, idf, top)
    
    if len(ranked_tweets) == 0:
        print("\n======================\nNo matches were found.")
    return ranked_tweets.loc[:, ranked_tweets.columns != 'terms'].reset_index(drop=True)

For each qquery, compute both rankings (tf-dif and diverse).

In [18]:
tfidf_queries_df = pd.DataFrame(columns = columns)
div_queries_df = pd.DataFrame(columns = columns)

for query in myQueries:
    result_tweets = submit_search(top, myIndex, df_tweets_clustered, query)
    
    # rank by tf-idf
    tf_idf_ranked_df = rank(top, result_tweets, myIndex, idf, query, "1") # Method "1" corresponds to tf-idf
    tf_idf_ranked_df['query'] = query
    tfidf_queries_df = pd.concat([tfidf_queries_df, tf_idf_ranked_df[columns]])
    
    # rank by divers
    div_ranked_df = divers_rank(query, result_tweets, myIndex, idf, top,tweet_id_to_cluster_dict , l)
    div_ranked_df['query'] = query
    div_queries_df = pd.concat([div_queries_df, div_ranked_df[columns]])


Save the results.

In [19]:
# save tf-idf
output_filename = os.path.join(output_dir, 'RQ2_a_nodiverse.tsv')
tfidf_queries_df.to_csv(output_filename, sep = '\t')

# save diverse
output_filename = os.path.join(output_dir, 'RQ2_a_diverse.tsv')
div_queries_df.to_csv(output_filename, sep = '\t')

In [26]:
tfidf_queries_df[tfidf_queries_df['query']=='trump']

,query,id,text,user,likes,retweets,hashtags,cluster,score
0,trump,1330165437414432768,tRump Vaccine for the tRump Virus,07834142-57cb-5655-8c9c-8e06deda5119,1450,98,[],7,6.906574
1,trump,1330164612348735497,Dabo Trump,7373df4f-6285-544d-bcc4-ddc920209b55,2,1,[],3,5.981268
2,trump,1330183654925070346,trumps words. It is what it is. 🤣,3101f384-5d0b-5fed-9e3a-4c4021649828,1,0,[],3,5.981268
3,trump,1330181291833241604,Trump flu,2e6f063a-3a05-595b-b476-30ccda8b054f,2123,415,[],3,5.981268
4,trump,1330180343614349321,@d22026c6-fe3c-5852-aa7f-e96bbf939fd2 Why trump did nothing . Trum...,1e428c40-33ec-500b-b389-53b79d8c162d,0,0,[],3,5.639194
5,trump,1330160117694803969,RT @ee21789c-1c0a-557b-b9f3-8b6bec5b5aa2: We Believe in...\n\n2021...,4f90797a-d685-55a2-8ede-c61fd7120e87,0,0,[],3,5.553468
6,trump,1330169081538949124,RT @ee21789c-1c0a-557b-b9f3-8b6bec5b5aa2: We Believe in...\n\n2021...,af181708-dfd2-5ed8-a7fe-aae72d285a48,0,0,[],3,5.553468
7,trump,1330180365089189898,RT @ee21789c-1c0a-557b-b9f3-8b6bec5b5aa2: We Believe in...\n\n2021...,57b180e3-c5de-557f-a232-95e3692b72cb,0,0,[],3,5.553468
8,trump,1330165482960392194,RT @f9b19cb1-403a-5b71-b369-c8eb3cd97120: Trump is twisted. He was...,75c62a96-5662-5dc3-92f6-6dded2e543c7,151,120,[],3,5.537578
9,trump,1330157019710562305,@02e6fb19-6e6e-5b3a-977b-14c8000d52d1 Does he really have Covid or...,f4d7ba3c-503c-5060-8489-8e3cc39aa4b9,0,0,[],3,5.100843


In [27]:
div_queries_df[div_queries_df['query']=='trump']

,query,id,text,user,likes,retweets,hashtags,cluster,score
0,trump,1330165437414432768,tRump Vaccine for the tRump Virus,07834142-57cb-5655-8c9c-8e06deda5119,1450,98,[],7,6.906574
1,trump,1330183654925070346,trumps words. It is what it is. 🤣,3101f384-5d0b-5fed-9e3a-4c4021649828,1,0,[],3,5.981268
2,trump,1330183695333003266,RT @fa75fa83-40e9-5478-80f7-6881a399c373: After meeting w/ Pres. T...,3bddad1d-62a7-59f5-930a-1420bf398d6e,1019,641,[],0,4.368101
3,trump,1330184229150470145,This will upset Trump more than a quarter of a million dead Americ...,d591ebc7-be3c-5cf0-b387-f9fe39799921,371,34,[],10,3.453287
4,trump,1330181198845534212,G20 summit on the pandemic and Trump is golfing. Ok bitch,6d4ad072-6769-53f3-b6fd-4495fee2f81a,0,0,[],11,3.197122
5,trump,1330183634532392960,@efbf20c9-674d-5add-9500-16ba9f92bab3 #butitsnotdumbass Trump didn...,02418b58-a098-524a-b8ec-f01091097cb8,0,0,[butitsnotdumbass],11,3.197122
6,trump,1330180371342888963,Trump has done everything in his power to keep this\n#TrumpsKiller...,1380348d-27a0-51a4-b796-90d74bfa8549,544,60,[TrumpsKillerCovidVirus],6,2.990634
7,trump,1330165427499118600,@c01b03d2-bfd1-5482-8b74-6792580daa6f @f91dc4ec-2fe0-505f-a048-da8...,6159500a-4101-5e7e-bea0-90362d88b04d,0,0,[],4,2.819597
8,trump,1330173916631998466,RT @1419f055-425b-5199-83fe-da3d98c37dcc: Trump and the Canadian C...,fc5aaac0-f76a-53c7-9cc2-92bd93184e60,0,0,[],11,2.674904
9,trump,1330164593268830209,RT @bb733d3a-cf28-55fe-b9f5-d80af94a2d84: All The Cities Trump Had...,a749fd93-ddd4-5d1a-b085-9ff73efa8114,0,0,"[WearAMaskSaveALife, COVID19]",1,2.674904


## Comparing the rankings
### RQ 2B

Measuring the coverage for both rankings:

In [28]:
myQueries = ['covid', 'mask', 'trump']

We are going to use the coverage function defined above.

In [29]:
for query in myQueries:
    result_tweets = submit_search(top, myIndex, df_tweets_clustered, query)
    
    # rank by tf-idf
    tf_idf_ranked_df = rank(top, result_tweets, myIndex, idf, query, "1") # Method "1" corresponds to tf-idf
    tf_idf_cov = coverage(list(tf_idf_ranked_df['cluster']))
    print("TF-IDF coverage for query '{}': {}.".format(query,tf_idf_cov))
    
    # rank by divers
    div_ranked_df = divers_rank(query, result_tweets, myIndex, idf, top,tweet_id_to_cluster_dict , l)
    div_cov = coverage(list(div_ranked_df['cluster']))
    print("Diverse ranking coverage for query '{}': {}.".format(query,div_cov))

TF-IDF coverage for query 'covid': 2.
Diverse ranking coverage for query 'covid': 12.
TF-IDF coverage for query 'mask': 1.
Diverse ranking coverage for query 'mask': 8.
TF-IDF coverage for query 'trump': 2.
Diverse ranking coverage for query 'trump': 11.


In order to compare the correlation of the rankings, we are going to use the Kendall's Tau for the Top-20, which is defined as:

    T = 2((number of concordant pairs) - (number of discordant parts))/n(n - 1)

In [30]:
def Kendall(rank1, rank2, n):
    num_con = 0 # concordant pairs count
    num_discon = 0 # disconcordant pairs count
    for i in range(n):
        if rank1[i] == rank2[i]: # if the tweet id in the i position is the same... 
            num_con += 1
        else:
            num_discon += 1 # if not
    T = 2*(num_con - num_discon)/(n*(n - 1))   
    return T

In [31]:
for query in myQueries:
    result_tweets = submit_search(top, myIndex, df_tweets_clustered, query)
    
    # rank by tf-idf
    tf_idf_ranked_df = rank(top, result_tweets, myIndex, idf, query, "1") # Method "1" corresponds to tf-idf
    
    # rank by divers
    div_ranked_df = divers_rank(query, result_tweets, myIndex, idf, top,tweet_id_to_cluster_dict , l)
    
    # compute kendall's T for the ranked tweet's ids
    T = Kendall(tf_idf_ranked_df['id'], div_ranked_df['id'], top)
    print("Kendall's T for query '{}': {}.".format(query,round(T,2)))

Kendall's T for query 'covid': -0.08.
Kendall's T for query 'mask': -0.11.
Kendall's T for query 'trump': -0.09.
